In [2]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [3]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # my cache path
vi_dir = '/pscratch/sd/j/jjpim/visual_inspection/' # vi image path

In [4]:
# load updated fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA_VI.fits') 
tf_fuji[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1.0,0.012960487467698754,0.3300000056318236,0.0,0.0
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1.0,0.012960487467685656,0.33000000563149007,0.0,0.0
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1.0,0.026313716979861984,0.6700000114340222,0.0,0.0


In [5]:
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA_VI.fits')
tf_iron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076,0.0,0.0567420462925582,0.599999979158402,0.0,1.0
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676,0.0,0.09457007715465947,0.9999999652681886,0.0,1.0
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771,0.0,0.024841917763806447,0.9999999684347345,0.0,0.0
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968,0.0,0.026555350833845817,0.8000000542794348,0.0,0.0
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061,0.0,0.015878717350385107,0.40000000619171416,0.0,0.0


In [6]:
tf_fuji['VELOCITY']=np.nan
tf_fuji['VELOCITY_REL']=np.nan
tf_iron['VELOCITY']=np.nan
tf_iron['VELOCITY_REL']=np.nan

In [7]:
# Filter fuji and iron for viable targets
filtered_tf_fuji = tf_fuji[tf_fuji['SKY_FIBER_DIST_R26'] < 0.001]
filtered_tf_iron = tf_iron[tf_iron['SKY_FIBER_DIST_R26'] < 0.001]
filtered_tf_fuji = filtered_tf_fuji[filtered_tf_fuji['ROT_CURVE'] == 1]
filtered_tf_iron = filtered_tf_iron[filtered_tf_iron['ROT_CURVE'] == 1]

# Find and print the maximum Z value
max_value_fuji = filtered_tf_fuji['Z'].max()
max_value_iron = filtered_tf_iron['Z'].max()
print("Maximum value in tf_fuji:", max_value_fuji)
print("Maximum value in tf_iron:", max_value_iron)

Maximum value in tf_fuji: 0.173630148670864
Maximum value in tf_iron: 1.62543750380985


In [8]:
#pull out all unique SGA ID's for iron and fuji
tf_fuji_rot = np.unique(filtered_tf_fuji[filtered_tf_fuji['ROT_CURVE']==1]['SGA_ID'])
tf_iron_rot = np.unique(filtered_tf_iron[filtered_tf_iron['ROT_CURVE']==1]['SGA_ID'])
#combine iron and fuji unique ID's, print final list 
tf_galaxies = np.unique(np.concatenate((tf_fuji_rot, tf_iron_rot)))
tf_galaxies = tf_galaxies.tolist()
print(tf_galaxies)

[123, 896, 1548, 1583, 1980, 2497, 5732, 5958, 5964, 6443, 8032, 9338, 9519, 9769, 10133, 10325, 10553, 11977, 11998, 12585, 12892, 13851, 13880, 14501, 15005, 16256, 16383, 18191, 19235, 19278, 19682, 20179, 20258, 20886, 21905, 23897, 25100, 25532, 25835, 26866, 26930, 28423, 28642, 29121, 29289, 30149, 31127, 31220, 32406, 32484, 32907, 33662, 34497, 35184, 35712, 36532, 36904, 36913, 37543, 39247, 39919, 42992, 43211, 46327, 50360, 54974, 56757, 57891, 58617, 61908, 71578, 73111, 79898, 81194, 81587, 88963, 90791, 91660, 92403, 92536, 93731, 98148, 98921, 98934, 100987, 102737, 103508, 103535, 104913, 105847, 106536, 107105, 108576, 108893, 109333, 110814, 111102, 113767, 114405, 114489, 115283, 116764, 117242, 117558, 118397, 122260, 122580, 122810, 124900, 125245, 125611, 126630, 127141, 127268, 128130, 128163, 128411, 128944, 129388, 130437, 130766, 133855, 135061, 135235, 136307, 139660, 149009, 149502, 153504, 154215, 156492, 158556, 158910, 159963, 166290, 170351, 171058, 171

In [19]:
# list of SGA IDs to loop through
sga_ids = [tf_galaxies]

# loop through each SGA_ID
for sga_id in sga_ids:
    targets_fuji = tf_fuji[tf_fuji['SGA_ID'] == sga_id]
    targets_iron = tf_iron[tf_iron['SGA_ID'] == sga_id]
    target_list = (vstack([targets_fuji, targets_iron]))
    
    #identify z values of targets
    z_targets=(target_list['Z'])
    
    #identify good centers
    fuji_centers = targets_fuji[targets_fuji['SKY_FIBER_DIST_R26'] < 0.001]
    fuji_centers = fuji_centers[fuji_centers['ZWARN']==0]
    fuji_good_centers = fuji_centers[fuji_centers['DELTACHI2']>25] 
    print(fuji_good_centers)
    
    iron_centers = targets_iron[targets_iron['SKY_FIBER_DIST_R26'] < 0.001]
    iron_centers = iron_centers[iron_centers['ZWARN']==0]
    iron_good_centers = iron_centers[iron_centers['DELTACHI2']>25] 
    print(iron_good_centers)

IndexError: Illegal key or index value for TableColumns object

In [40]:
#list of good centers in fuji for one galaxy found by SGA ID above
fuji_centers = targets_fuji[targets_fuji['SKY_FIBER_DIST_R26'] < 0.001]
fuji_centers = fuji_centers[fuji_centers['ZWARN']==0]
fuji_good_centers = fuji_centers[fuji_centers['DELTACHI2']>25] 
fuji_good_centers

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VELOCITY_REL
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39627758149572214,215.992684704008,-1.12847885784913,25954,sv3,0.0344118495024228,3.99350519839902e-06,0,2087.82850825787,fuji/healpix/sv3/bright/259/25954/redrock-sv3-bright-25954.fits,TFT,218856,215.9926847040083,-1.1284788578491263,1.0,0.0,0.0,1.0,0.0,nan,nan


In [41]:
#list of good centers in iron for one galaxy found by SGA ID above
iron_centers = targets_iron[targets_iron['SKY_FIBER_DIST_R26'] < 0.001]
iron_centers = iron_centers[iron_centers['ZWARN']==0]
iron_good_centers = iron_centers[iron_centers['DELTACHI2']>25] 
iron_good_centers

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VELOCITY_REL
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39627758149572214,215.992684704008,-1.12847885784913,25954,main,0.034381636875159,3.2100483776182e-06,0,3770.57050712407,iron/healpix/main/bright/259/25954/redrock-main-bright-25954.fits,TFT,218856,215.9926847040083,-1.1284788578491263,1.0,0.0,0.0,1.0,0.0,nan,nan
39627758149572214,215.992684704008,-1.12847885784913,25954,sv3,0.0344116677276749,3.99256805928159e-06,0,2104.89893661439,iron/healpix/sv3/bright/259/25954/redrock-sv3-bright-25954.fits,TFT,218856,215.9926847040083,-1.1284788578491263,1.0,0.0,0.0,1.0,0.0,nan,nan


In [83]:
#defining terms
z_fuji = fuji_good_centers['Z']
z_iron = iron_good_centers['Z']
z_fuji_err = fuji_good_centers['ZERR']
z_iron_err = iron_good_centers['ZERR']

# calculating the weights
w_fuji = 1 / (z_fuji_err**2)
w_iron = 1 / (z_iron_err**2)
w = np.sum(w_fuji + w_iron)

# calculating the weighted average z naught
w1 = (w_fuji * z_fuji)
w2 = (w_iron * z_iron)
z_o = (np.sum(w1 + w2)) / w

print(z_o)

0.03440152846183661


In [84]:
#define z naught error
z_o_err = np.sum(1 / ((w_fuji)**0.5) + 1 / ((w_iron)**0.5))
print(z_o_err)

1.518962683369783e-05


In [85]:
#define z error for all targets
for TARGETID in target_list:
        z_err=target_list['ZERR']
print(z_err)

        ZERR        
--------------------
0.000150406289231301
2.69656859568662e-06
3.99350519839902e-06
 4.5506520619317e-06
2.31625604106805e-06
8.70998492851394e-05
 3.2100483776182e-06
3.99256805928159e-06
 2.0116123095721e-06


In [87]:
#find delta z from z naught and z values of all viable targets 
delta_z = z_targets - z_o
print(delta_z)

           Z           
-----------------------
     1.4850983531210933
 0.00022350385804048578
 1.0321040586187824e-05
 -0.0002551955616839144
 -0.0002594837768414085
     1.5221652356376334
-1.9891586677610318e-05
  1.013926583828817e-05
 0.00025189330753108696


In [104]:
#use delta z to find velocity
c = 299792.458
velocity = c*delta_z
print(velocity)

         Z         
-------------------
  445221.2856539245
  67.00477097444029
 3.0941701264510084
  -76.5057047079113
 -77.79127927040932
  456333.6574739553
-5.9633476636008504
  3.039675427975841
  75.51571381849446


In [106]:
#put results into column on tables - don't run yet      
for i in range(len(target_list)):
    if target_list['TARGETID'][i] in tf_fuji:
        tf_fuji['VELOCITY'][i] = velocity
        
for i in range(len(target_list)):
    if target_list['TARGETID'][i] in tf_iron:
        tf_iron['VELOCITY'][i] = velocity
        
tf_fuji

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VELOCITY_REL
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0,nan,nan
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0,nan,nan
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1.0,0.012960487467698754,0.3300000056318236,0.0,0.0,nan,nan
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1.0,0.012960487467685656,0.33000000563149007,0.0,0.0,nan,nan
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1.0,0.026313716979861984,0.6700000114340222,0.0,0.0,nan,nan
1092744369930241,194.325065734878,27.4857950068364,10378,sv3,0.34112572843082,7.87536530522298e-05,4,5.80425057094544,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3250657348779,27.48579500683636,1.0,0.026313716979877486,0.6700000114344169,0.0,0.0,nan,nan
1079550230396930,194.312068074608,27.4798777913963,10378,sv3,1.43883246626074,0.000118079246392633,4,1.41281921416521,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.3120680746076,27.479877791396284,1.0,0.039274204447592366,1.0000000170666512,0.0,0.0,nan,nan
1092744369930242,194.312068074608,27.4798777913963,10378,sv3,0.141836800089955,6.14844128636576e-05,4,8.01571118086576,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3120680746076,27.479877791396284,1.0,0.039274204447592366,1.0000000170666512,0.0,0.0,nan,nan
1080141245579266,268.972484124775,62.5902888954075,10207,sv3,1.25885927887654,0.000161730072763419,4,4.86104696989059,fuji/healpix/sv3/bright/102/10207/redrock-sv3-bright-10207.fits,EXT,674212,268.972484124775,62.59028889540747,1.0,0.023926091509660182,0.6699999854676792,0.0,1.0,nan,nan


In [91]:
#find v error for each target
v_error_1 = ((z_o_err**2) + (z_err**2))
v_err=c*(v_error_1**0.5)
print(v_err)
# put v error into new column for each table

       ZERR       
------------------
 45.32003014238856
 4.624936304398248
 4.708487005395994
  4.75370264521897
 4.606375313036527
26.505974336116157
 4.654312175816296
 4.708415577035106
 4.593495012761475


In [ ]:
#calculate velocity relative to galaxy